In [ ]:
# =====================================
# 1) Leer Silver
# =====================================
from pyspark.sql import functions as F

df_silver = spark.table("silver_personas")

print("Registros en Silver:", df_silver.count())
display(df_silver)

In [ ]:
# =====================================
# 2) Derivar atributos de segmentación y construir dimensión
#    - Surrogate Key: entero estable a partir de BK usando xxhash64
#    - Segmentos de Frecuencia / Ticket
# =====================================

# Surrogate Key: entero positivo a partir de la BK
sk = F.abs(F.xxhash64(F.col("BK_CLIENTE"))) + F.lit(1)

# Segmentos (puedes ajustar los umbrales a tu negocio)
seg_frec = (
    F.when(F.col("FrecuenciaVisita") >= 12, "ALTA")
     .when(F.col("FrecuenciaVisita") >= 6, "MEDIA")
     .when(F.col("FrecuenciaVisita") >= 1, "BAJA")
     .otherwise("NULA")
)

seg_ticket = (
    F.when(F.col("TicketPromedio") >= 60, "ALTO")
     .when(F.col("TicketPromedio") >= 30, "MEDIO")
     .when(F.col("TicketPromedio") > 0,   "BAJO")
     .otherwise("SIN_DATO")
)

dim_cliente = (
    df_silver.select(
        sk.alias("SK_CLIENTE"),                 # surrogate key
        F.col("BK_CLIENTE"),                    # clave de negocio
        F.col("IdCliente").alias("ID_ORIGEN"),  # id fuente si aplica
        F.col("Documento"),
        F.col("Email"),
        F.col("Telefono"),
        F.col("Nombre"),
        F.col("Apellido"),
        F.col("Genero"),
        F.col("Edad"),
        F.col("Pais"),
        F.col("Ciudad"),
        F.col("MarcaPreferida"),
        F.col("FechaRegistro"),
        F.col("UltimaCompra"),
        F.col("FrecuenciaVisita"),
        F.col("TicketPromedio"),
        F.col("PuntosAcumulados"),
        seg_frec.alias("SegmentoFrecuencia"),
        seg_ticket.alias("SegmentoTicket"),
        F.col("EstadoCliente"),
        F.col("Observaciones")
    )
    # Type-1: sobreescribe en cada refresco; si quisieras SCD2, aquí harías el merge con vigencias
)

display(dim_cliente)

In [ ]:
# =====================================
# 3) Guardar GOLD – Dimensión Cliente
# =====================================

(dim_cliente.write
 .format("delta")
 .mode("overwrite")
 .saveAsTable("gold_dim_cliente"))

print("🏅 GOLD creada: gold_dim_cliente (dimensión cliente, Type-1)")


In [ ]:
-- Ejemplo de agregación (analítica): Clientes por Marca x Ciudad

In [ ]:
# =====================================
# 4) Agregado Gold (ejercicio analítico)
#    - Cuenta de clientes por MarcaPreferida y Ciudad
#    - TicketPromedio medio y Puntos promedio por grupo
# =====================================


gold_agg = (
    dim_cliente
    .groupBy("MarcaPreferida", "Ciudad")
    .agg(
        F.countDistinct("SK_CLIENTE").alias("Clientes"),
        F.round(F.avg("TicketPromedio"),2).alias("TicketPromedioProm"),
        F.round(F.avg("PuntosAcumulados"),2).alias("PuntosProm")
    )
    .orderBy(F.col("Clientes").desc_nulls_last())
)

display(gold_agg)



In [ ]:
# =====================================
# 5) Guardar GOLD – Agregado
# =====================================

(gold_agg.write
 .format("delta")
 .mode("overwrite")
 .saveAsTable("gold_clientes_marca_ciudad"))

print("🏅 GOLD creada: gold_clientes_marca_ciudad (agregado analítico)")
